In [4]:
import pandas as pd

In [5]:
# Merging clients and dispositions
clients = pd.read_csv("data/client.csv", sep=";")

disps = pd.read_csv("data/disp.csv", sep=";")

clients_disps = pd.merge(clients, disps, on="client_id")

In [6]:
clients_disps

,client_id,birth_number,district_id,disp_id,account_id,type
0,1,706213,18,1,1,OWNER
1,2,450204,1,2,2,OWNER
2,3,406009,1,3,2,DISPONENT
3,4,561201,5,4,3,OWNER
4,5,605703,5,5,3,DISPONENT
...,...,...,...,...,...,...
5364,13955,456030,1,13647,11349,OWNER
5365,13956,430406,1,13648,11349,DISPONENT
5366,13968,680413,61,13660,11359,OWNER
5367,13971,626019,67,13663,11362,OWNER
